In [11]:
from confluent_kafka import Consumer
import uuid
import time

In [6]:
bootstrap_server = "localhost:9092"
msg_count = 1000000

# client = KafkaClient(hosts=bootstrap_server)
# topic = client.topic[b'test']
# print(topic.earliest_available_offsets())
# print(topic.latest_available_offsets())

consumer_timings = {}

def calculate_thoughput(timing, n_messages=1000000, msg_size=100):
    print("Processed {0} messsages in {1:.2f} seconds".format(n_messages, timing))
    print("{0:.2f} MB/s".format((msg_size * n_messages) / timing / (1024*1024)))
    print("{0:.2f} Msgs/s".format(n_messages / timing))

In [21]:
def consumer_performance() :
    topic = "test"
    msg_consumed_count = 0
    conf = {
        "bootstrap.servers": bootstrap_server,
        "group.id": uuid.uuid1(),
        "session.timeout.ms": 6000,
        "default.topic.config": {
            "auto.offset.reset": "earliest"
        }
    }

    consumer = Consumer(**conf)
    consumer_start = time.time()

    consumer.subscribe([topic])

    while True :
        msg = consumer.poll(1)
        print(msg)
        if msg :
            msg_consumed_count += 1
        if msg_consumed_count >= msg_count :
            break
    
    consumer_timing = time.time() - consumer_start
    consumer.close()
    return consumer_timing


            

In [ ]:
_ = consumer_performance() # Warm cache
consumer_timings['confluent_kafka_consumer'] = consumer_performance()
calculate_thoughput(consumer_timings['confluent_kafka_consumer'])

In [41]:
topic = "test"
msg_consumed_count = 0
conf = {
    "bootstrap.servers": bootstrap_server,
    "group.id": uuid.uuid1(),
    "session.timeout.ms": 6000,
    "default.topic.config": {
        "auto.offset.reset": "earliest"
    }
}

consumer = Consumer(**conf)
consumer_start = time.time()

# for c in consumer :
#     print(c)

In [39]:
from confluent_kafka import KafkaError, KafkaException
import sys
running = True

def basic_consume_loop(consumer, topics):
    try:
        consumer.subscribe(topics)

        while running:
            msg = consumer.poll(timeout=1.0)
            if msg is None: continue

            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                print(msg.value())
                # msg_process(msg)
    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

def shutdown():
    running = False

In [ ]:
basic_consume_loop(consumer, ["test2"])